In [1]:
import os
import numpy as np
from alphai_watson.datasource.brainwaves import BrainwavesDataSource
from alphai_watson.performance import GANPerformanceAnalysis
from alphai_watson.transformer import NullTransformer

from alphai_rickandmorty_oracle.detective import RickAndMortyDetective

/opt/anaconda/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/opt/anaconda/envs/ai/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Enabling weight norm
Uppercase local vars:
	BATCH_SIZE: 50
	CRITIC_ITERS: 5
	DEFAULT_FIT_EPOCHS: 1000
	DEFAULT_LEARN_RATE: 0.0001
	DEFAULT_TRAIN_ITERS: 5000
	DEFAULT_Z_DIM: 128
	DIAGNOSIS_LEARN_RATE: 0.01
	DIM: 64
	DISC_FILTER_SIZE: 5
	LAMBDA: 10
	LAMBDA_2: 2.0
	OUTPUT_DIM: 784


In [2]:
import h5py
filename = os.path.join('../../tests/resources', 'brainwaves_normal_sample_1.hd5')
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])
data

Keys: KeysView(<HDF5 file "brainwaves_normal_sample_1.hd5" (mode r)>)


['DATA', 'SAMPLE_RATE']

In [13]:
test_data_file = os.path.join('../../tests/resources', 'brainwaves_normal_sample_1.hd5')
n_sensors = 16
n_timesteps = 784 // n_sensors

train_data_source = BrainwavesDataSource(source_file=test_data_file,
                                         transformer=NullTransformer(number_of_timesteps=n_timesteps, 
                                                                     number_of_sensors=n_sensors))

train_data = train_data_source.get_train_data('NORMAL')

train_data.data.shape


DEBUG:root:Start file parsing
DEBUG:root:end file parsing


(4892, 8, 98)

In [6]:
train_data.data.shape

(29970, 16, 8)

In [ ]:
detective = RickAndMortyDetective(model_configuration={
    'batch_size': 64,
    'output_dimensions': 784,
    'train_iters': 200,
    'plot_save_path' : './'
})

detective.train(train_data)